# Reconhecimento Facial em Câmeras de Segurança aplicado em Cidades Inteligentes e Segurança Pública

Este notebook apresenta o processo de construção de um modelo de **reconhecimento facial** baseado no *dataset* *SCface*.

Neste notebook, estão presentes as etapas de:
- 1. Preparação do conjunto de dados
- 2. Detecção facial com *MTCNN*
- 3. Extração de características com *FaceNet e ArcFace*
- 4. Identificação facial através da comparação de *embeddings*

# 0. Importando bibliotecas e dependências, além de fazer device-agnostic code

In [ ]:
!nvidia-smi

Wed Aug 13 13:38:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   28C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Montar Google Colab para ter acesso ao dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Instalando bibliotecas necessárias
!pip install facenet-pytorch --no-deps
!pip install -U albumentationsx
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.24
    Uninstalling albucore-0.0.24:
      Successfully uninstalled albucore-0.0.24
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires albucore==0.0.24, but you have albucore 0.0.33 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━

In [ ]:
# Configurando o repositório do CodeFormer
!rm -rf CodeFormer # Remove a pasta antiga se existir
!git clone https://github.com/sczhou/CodeFormer
%cd CodeFormer

# Instalando dependencias
!pip install -r requirements.txt
!python basicsr/setup.py develop

# Baixando modelos pré-treinados
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

%cd ..

Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 614 (delta 203), reused 176 (delta 176), pack-reused 321 (from 2)
Receiving objects: 100% (614/614), 17.31 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/CodeFormer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.8 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        *********************************************************************

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics.pairwise import cosine_similarity

import os
import re
import zipfile
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from timeit import default_timer as timer
import copy
import time
from scipy import stats

from facenet_pytorch import InceptionResnetV1, MTCNN

In [ ]:
print(f"Versão do PyTorch: {torch.__version__}")

Versão do PyTorch: 2.6.0+cu124


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Variáveis globais
SEED = 42
EPOCHS = 30
BATCH_SIZE = 32
IMG_SIZE = 160
LEARNING_RATE = 0.00005

# 1. Pegar os dados do SCface

## 1.1 Descompactando o dataset e definindo os caminhos para as pastas do dataset

In [ ]:
# Caminho zip do drive
zip_path = '/content/drive/MyDrive/PIBIC 2024-2025/SCface.zip'

# Caminho para qual iremos extrair
extract_path = Path("/content/scface_data")

# Extrai se o diretório de destino não existir
if not extract_path.exists():
  print(f"Criando diretório e descompactando em: {extract_path}")

  os.makedirs(extract_path, exist_ok=True)

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

  print("Dataset extraído com sucesso.")
else:
  print(f"Diretório '{extract_path}' já existe. Extração pulada.")

Criando diretório e descompactando em: /content/scface_data
Dataset extraído com sucesso.


In [ ]:
# Definindo os caminhos para o dataset
SCface_path = extract_path / "SCface_database"

SCFACE_MUGSHOT_FRONTAL_PATH = SCface_path / "mugshot_frontal_cropped_all"
SCFACE_MUGSHOT_ROTATION_PATH= SCface_path / "mugshot_rotation_all"
SCFACE_PROBE_PATH = SCface_path / "surveillance_cameras_all"

# Caminho para a pasta onde as novas imagens restauradas serão salvas
SCFACE_RESTORED_PROBE_PATH = SCface_path / "surveillance_cameras_all_restored"

In [ ]:
print("Carregando e combinando imagens da galeria frontal e de rotação...")

frontal_files = list(Path(SCFACE_MUGSHOT_FRONTAL_PATH).glob('*.jpg'))

rotation_files = list(Path(SCFACE_MUGSHOT_ROTATION_PATH).glob('*.jpg'))

# Lista combinada de todas as imagens frontais e todas as imagens de rotação
all_gallery_filepaths = frontal_files + rotation_files

print(f"Total de {len(all_gallery_filepaths)} imagens de alta qualidade encontradas para treino/validação.")

Carregando e combinando imagens da galeria frontal e de rotação...
Total de 1170 imagens de alta qualidade encontradas para treino/validação.


# 2. Data augmentation das imagens

In [ ]:
# Transforms de treino
train_transform = A.Compose([
    # O resize será aplicado após pré-processamento do MTCNN
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.8),
    A.GaussNoise(p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.3),
        A.MedianBlur(blur_limit=3, p=0.2),
        A.GaussianBlur(p=0.3),
    ], p=0.5),
    A.GridDistortion(p=0.2),
    A.CoarseDropout(
        num_holes_range=(1, 8),
        hole_height_range=(8, 16),
        hole_width_range=(8, 16),
        fill=0,
        p=0.5
    ),
    A.Rotate(limit=10, p=0.5),
    # Normaliza os pixels da imagem para o intervalo [-1, 1]
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2(),
])

# Transforms da avaliação
val_test_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2(),
])

# 3. Criando a classe Dataset e os DataLoaders para o SCface

Aqui, a gallery (galeria) representa as imagens que representam as fotos de rostos, "estilo mugshot" e o probe representa as imagens de câmeras de segurança.

## 3.1 Declaração da MTCNN e Restauração das imagens com CodeFormer

In [ ]:
# Iniciando MTCNN
mtcnn = MTCNN(image_size=IMG_SIZE,
              margin=0,
              keep_all=False,
              device=device)

In [ ]:
# Criando diretorio para imagens restauradas, caso nao exista
if not os.path.exists(SCFACE_RESTORED_PROBE_PATH):
  os.makedirs(SCFACE_RESTORED_PROBE_PATH, exist_ok=True)

In [ ]:
# Comando para rodar a inferência do CodeFormer
# -w: peso da fidelidade à imagem original
# --bg_upsampler: para melhorar o fundo, totalmente opcional
# --face_upsample: garante que a face seja redimensionada
!python CodeFormer/inference_codeformer.py -w 0.7 --input_path "{SCFACE_PROBE_PATH}" --output_path "{SCFACE_RESTORED_PROBE_PATH}" --bg_upsampler realesrgan --face_upsample

A saída de streaming foi truncada nas últimas 5000 linhas.
	detect 1 faces
	Input is a 16-bit image
[809/2860] Processing: 037_cam6_2.jpg
Grayscale input: True
	detect 1 faces
	Input is a 16-bit image
[810/2860] Processing: 037_cam6_3.jpg
Grayscale input: True
	detect 1 faces
	Input is a 16-bit image
[811/2860] Processing: 037_cam7_1.jpg
Grayscale input: True
	detect 1 faces
[812/2860] Processing: 037_cam7_2.jpg
Grayscale input: True
	detect 1 faces
[813/2860] Processing: 037_cam7_3.jpg
Grayscale input: True
	detect 1 faces
[814/2860] Processing: 037_cam8.jpg
Grayscale input: True
	detect 1 faces
	Input is a 16-bit image
[815/2860] Processing: 038_cam1_1.jpg
	detect 1 faces
[816/2860] Processing: 038_cam1_2.jpg
	detect 1 faces
[817/2860] Processing: 038_cam1_3.jpg
	detect 1 faces
[818/2860] Processing: 038_cam2_1.jpg
	detect 1 faces
[819/2860] Processing: 038_cam2_2.jpg
	detect 1 faces
[820/2860] Processing: 038_cam2_3.jpg
	detect 1 faces
[821/2860] Processing: 038_cam3_1.jpg
	detect 1

## 3.2 Classe do Dataset

In [ ]:
# Classe de Dataset para o SCface
class ScfaceDatasetWithMTCNN(Dataset):
  """Classe que representa uma adaptação de Dataset do Pytorch para o SCface e o
  uso da MTCNN. Esta classe utiliza a arquitetura MTCNN para detecção e alinha
  mento.
  """

  def __init__(self, filepaths, labels, transform=None, mtcnn_model=None):
    """Inicializa o Dataset adaptado para o SCface com suporte à detecção facial
    via MTCNN.

    Args:
      filepaths (list[str]): Lista de caminhos das imagens.
      labels (list[str]): Lista de rótulos correspondentes a cada
      imagem.
      transform (albumentations.Compose, opcional): Conjunto de transformações
      para as imagens (padrão é None).
      mtcnn_model (facenet_pytorch.MTCNN, opcional): Instância do modelo MTCNN
      (padrão é None).
    """
    self.filepaths, self.labels = filepaths, labels
    self.transform = transform
    self.mtcnn = mtcnn_model

  def __len__(self):
    """Returns:
        int: Número total de amostras no dataset."
    """
    return len(self.filepaths)

  def __getitem__(self, idx):
    """Obtém a amostra com índice "idx" indicado.

    Args:
      idx (int): Índice de uma imagem

    Returns:
      tuple (torch.tensor, int or str, str): img_tensor, label, str(img_path)
      ou None, caso haja algum erro
    """
    img_path, label = self.filepaths[idx], self.labels[idx]

    try:
      img_pil = Image.open(img_path).convert("RGB")
    except Exception as e:
      print(f"Erro ao carregar a imagem: {img_path}. Erro: {e}")

      return None

    # Detecção e alinhamento com MTCNN
    face_tensor = self.mtcnn(img_pil)

    if face_tensor is None:
      return None

    # O tensor do MTCNN está normalizado em [-1, 1]. Para Albumentations,
    # vamos convertê-lo de volta para uma imagem NumPy [0, 255]
    face_tensor = (face_tensor + 1) / 2.0

    img_np = face_tensor.permute(1, 2, 0).cpu().numpy() * 255.0

    img_np = img_np.astype(np.uint8)

    # Aplicar as transformações da Albumentations
    if self.transform:
      augmented = self.transform(image=img_np)

      img_tensor = augmented['image']

    return img_tensor, label, str(img_path)

In [ ]:
# Função Collate para lidar com amostras nas quais nenhum rostos é detectado
def collate_fn_skip_none(batch):
  """ Função Collate para o Dataloader que ignora amostras inválidas, aquelas em
  que nenhum rosto é detectado.

  Args:
    batch (list): Lista de amostras.

  Returns:
    tuple or None:
      - Se houver amostras válidas: retorna o batch processado pelo
        `torch.utils.data.dataloader.default_collate`.
      - Se todas as amostras forem inválidas: retorna `(None, None, None)`.
  """
  # Filtra qualquer amostra na qual o MTCNN falhou
  batch = [item for item in batch if item is not None]

  if not batch: # Se o batch inteiro for None
    return None, None, None

  # Desempacota o batch filtrado e o retorna
  return torch.utils.data.dataloader.default_collate(batch)

## 3.3 Criação dos Datasets e DataLoaders

In [ ]:
# Criar o mapa de rótulos unificado a partir da galeria
# A gallery geralmente contém todas as identidades que podem aparecer nas sondas.
print("Criando mapa de rótulos unificado a partir da galeria...")

all_ids_str = sorted(list(set([re.match(r'(\d+)', f.name).group(1) for f in all_gallery_filepaths])))

unified_label_map = {id_str: i for i, id_str in enumerate(all_ids_str)}

NUM_SCFACE_CLASSES = len(unified_label_map)

print(f"Mapa unificado criado com {NUM_SCFACE_CLASSES} identidades.")

# Carregar os dados da galeria
all_gallery_labels = [unified_label_map[re.match(r'(\d+)', f.name).group(1)] for f in all_gallery_filepaths]

print(f"Número total de amostras na galeria antes do filtro: {len(all_gallery_labels)}")

# Dividir a galeria em treino e validação (80/20) de forma estratificada
print("Dividindo dados de treino e validação...")
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)

train_indices, val_indices = next(splitter.split(all_gallery_filepaths, all_gallery_labels))

Criando mapa de rótulos unificado a partir da gallery...
Mapa unificado criado com 130 identidades.
Número total de amostras na galeria antes do filtro: 1170
Dividindo dados de treino e validação...


In [ ]:
# Criar os datasets de treino e validação
train_dataset = ScfaceDatasetWithMTCNN([all_gallery_filepaths[i] for i in train_indices],
                                       [all_gallery_labels[i] for i in train_indices],
                                       transform=train_transform,
                                       mtcnn_model=mtcnn)

val_dataset = ScfaceDatasetWithMTCNN([all_gallery_filepaths[i] for i in val_indices],
                                     [all_gallery_labels[i] for i in val_indices],
                                     transform=val_test_transform,
                                     mtcnn_model=mtcnn)

In [ ]:
# Criar os DataLoaders de treino e validação
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          collate_fn=collate_fn_skip_none)

val_loader = DataLoader(val_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        collate_fn=collate_fn_skip_none)

print(f"DataLoaders de treino ({len(train_dataset)} imgs) e validação ({len(val_dataset)} imgs) criados.")

DataLoaders de treino (936 imgs) e validação (234 imgs) criados.


In [ ]:
# Criar o DataLoader de teste (sondas)
print("Criando DataLoader de teste (sondas)...")
probe_filepaths_all = list(Path("/content/scface_data/SCface_database/surveillance_cameras_all_restored/restored_faces").glob('*.png'))

probe_labels_all = [unified_label_map.get(re.match(r'(\d+)', p.name).group(1)) for p in probe_filepaths_all]

# Filtramos as imagens de sonda que não têm uma identidade correspondente na galeria
probe_data_filtered = [(fp, lbl) for fp, lbl in zip(probe_filepaths_all, probe_labels_all) if lbl is not None]
probe_filepaths, probe_labels = zip(*probe_data_filtered)

# Criamos o dataset e o dataloader apenas com as sondas válidas
probe_dataset = ScfaceDatasetWithMTCNN(list(probe_filepaths),
                                       list(probe_labels),
                                       transform=val_test_transform,
                                       mtcnn_model=mtcnn)

probe_loader = DataLoader(probe_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          collate_fn=collate_fn_skip_none)

# Loader para a galeria completa
gallery_dataset_for_eval = ScfaceDatasetWithMTCNN(all_gallery_filepaths,
                                                  all_gallery_labels,
                                                  transform=val_test_transform,
                                                  mtcnn_model=mtcnn)

gallery_loader_for_eval = DataLoader(dataset=gallery_dataset_for_eval,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False,
                                     collate_fn=collate_fn_skip_none)

print(f"DataLoader de teste ({len(probe_loader)} imgs) criado.")
print(f"DataLoader de teste ({len(gallery_loader_for_eval)} imgs) criado.")

Criando DataLoader de teste (sondas)...
DataLoader de teste (88 imgs) criado.
DataLoader de teste (37 imgs) criado.


# 4. Treinamento do Modelo

In [ ]:
# Classe do ArcFace
class ArcFace(nn.Module):
  """Classe que representa a implementação do ArcFace.
  """

  def __init__(self, in_features, out_features, s=30.0, m=0.50):
    """Implementação do cabeçalho ArcFace para classificação com margem angular.

    Args:
      in_features (int): Dimensão dos embeddings de entrada.
      out_features (int): Número de classes.
      s (float, opcional): Fator de escala aplicado aos logits (padrão: 30.0).
      m (float, opcional): Margem angular aplicada à classe correta
      (padrão: 0.50).
    """

    super(ArcFace, self).__init__()

    self.in_features, self.out_features, self.s, self.m = in_features, out_features, s, m

    self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))

    nn.init.xavier_uniform_(self.weight)


  def forward(self, emb, lbl):
    """
    Executa a passagem direta do ArcFace.

    Args:
      emb (torch.Tensor): Tensor de embeddings normalizados de tamanho
          (batch_size, in_features).
      lbl (torch.Tensor): Tensor de rótulos de classe de tamanho (batch_size,).

    Returns:
      torch.Tensor: Logits com margem angular aplicada,
      prontos para função de perda.
    """
    cosine = F.linear(F.normalize(emb), F.normalize(self.weight))

    theta, one_hot = torch.acos(torch.clamp(cosine, -1, 1)), torch.zeros_like(cosine)

    one_hot.scatter_(1, lbl.view(-1, 1).long(), 1)

    output = self.s * ((one_hot * torch.cos(theta + self.m)) + ((1 - one_hot) * cosine))

    return output


# Classe do Modelo com ArcFace
class FaceRecognitionModelWithArcFace(nn.Module):
  """Modelo de reconhecimento facial que utiliza InceptionResnetV1 com ArcFace.
  """

  def __init__(self, num_classes, embedding_size=512):
    """Inicizaliza o modelo

    Args:
      num_classes (int): Número de classes (rótulos/labels)
      embedding_size: Tamanho dos embeddings gerados pelo modelo (padrão: 512).
    """

    super(FaceRecognitionModelWithArcFace, self).__init__()

    self.backbone = InceptionResnetV1(pretrained='vggface2', classify=False)

    self.head = ArcFace(in_features=embedding_size, out_features=num_classes)


  def forward(self, image, label):
    """Executa a passagem direta pelo modelo de reconhecimento facial

    Args:
      image (torch.Tensor): Batch da imagem, no formato (B, C, H, W).
      label (torch.Tensor): Tensor de rótulos de classe.

    Returns:
      torch.Tensor: Logits com margem angular aplicada pelo ArcFace.
    """

    embedding = self.backbone(image)

    logits = self.head(embedding, label)

    return logits

In [ ]:
# Inicializando modelo
print("Inicializando novo modelo para fine-tuning no Scface...")

scface_model = FaceRecognitionModelWithArcFace(num_classes=NUM_SCFACE_CLASSES).to(device)

Inicializando novo modelo para fine-tuning no Scface...


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
# Declarando otimizador e loss
optimizer = optim.Adam(scface_model.parameters(), lr=LEARNING_RATE)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
print("Iniciando o processo de treinamento...")

best_val_loss = float('inf')

best_model_state = None

patience_counter = 0

early_stopping_patience = 10 # O treino para se a Val Loss não melhorar por 10 épocas seguidas

start_time_total_train = timer()

for epoch in range(EPOCHS):
    # Fase de Treinamento
    scface_model.train()

    running_loss_train = 0.0

    running_acc_train = 0.0

    train_batches = 0

    loop_train = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS} [Treino]", leave=False)

    for images, labels, _ in loop_train:
      images, labels = images.to(device), labels.to(device)

      # 1. Forward pass com imagem e rótulo para o ArcFace
      logits = scface_model(images, labels)

      # 2. Calcular a perda
      loss = loss_fn(logits, labels)

      # 3. Zerar o gradiente do otimizador
      optimizer.zero_grad()

      # 4. Backpropagation
      loss.backward()

      # 5. Step do otimizador
      optimizer.step()

      # Acumular métricas
      running_loss_train += loss.item()
      preds = torch.argmax(logits, dim=1)
      running_acc_train += (preds == labels).float().mean().item()
      train_batches += 1
      loop_train.set_postfix(loss=loss.item())

    avg_loss_train = running_loss_train / train_batches

    avg_acc_train = running_acc_train / train_batches

    # Fase de Avaliação
    scface_model.eval()

    running_loss_val = 0.0

    running_acc_val = 0.0

    val_batches = 0

    with torch.inference_mode():
      loop_val = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{EPOCHS} [Avaliação]", leave=False)

      for images, labels, _ in loop_val:
        images, labels = images.to(device), labels.to(device)

        # 1. Forward pass com imagem e rótulo
        logits = scface_model(images, labels)

        # 2. Calcular perda
        loss = loss_fn(logits, labels)
        running_loss_val += loss.item()

        # Calcular acurácia
        preds = torch.argmax(logits, dim=1)
        running_acc_val += (preds == labels).float().mean().item()
        val_batches += 1
        loop_val.set_postfix(val_loss=loss.item())

    avg_loss_val = running_loss_val / val_batches
    avg_acc_val = running_acc_val / val_batches

    print(f"Epoch {epoch+1}/{EPOCHS} | Treino Loss: {avg_loss_train:.4f}, Treino Acc: {avg_acc_train:.4f} | "
          f"Val Loss: {avg_loss_val:.4f}, Val Acc: {avg_acc_val:.4f}")

    # Lógica para salvar o melhor modelo e Early Stopping
    if avg_loss_val < best_val_loss:
      best_val_loss = avg_loss_val
      best_model_state = copy.deepcopy(scface_model.state_dict())
      patience_counter = 0
      print(f"  Novo melhor modelo encontrado! Val Loss: {best_val_loss:.4f}. Salvando estado...")
    else:
      patience_counter += 1

    if patience_counter >= early_stopping_patience:
      print(f"  Early stopping na Época {epoch + 1} pois a perda de validação não melhorou por {early_stopping_patience} épocas.")
      break

end_time_total_train = timer()

print(f"\nTempo total de treinamento: {end_time_total_train - start_time_total_train:.3f} segundos")

if best_model_state:
  print("Treinamento concluído. O melhor estado do modelo foi salvo na memória.")

  # Salvar o modelo final no disco
  torch.save(best_model_state, 'scface_best_model.pth')
  print("Melhor modelo salvo em 'scface_best_model.pth'")
else:
  print("Treinamento concluído, mas nenhum modelo foi salvo.")

Iniciando o processo de treinamento...


Epoch 1/30 [Treino]:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1/30 [Avaliação]:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/30 | Treino Loss: 19.6565, Treino Acc: 0.0000 | Val Loss: 18.5057, Val Acc: 0.0000
  Novo melhor modelo encontrado! Val Loss: 18.5057. Salvando estado...


Epoch 2/30 [Treino]:   0%|          | 0/30 [00:00<?, ?it/s]

...

Epoch 29/30 [Avaliação]:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 29/30 | Treino Loss: 1.4819, Treino Acc: 0.7534 | Val Loss: 0.6311, Val Acc: 0.8820


Epoch 30/30 [Treino]:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 30/30 [Avaliação]:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 30/30 | Treino Loss: 1.2385, Treino Acc: 0.7985 | Val Loss: 0.3977, Val Acc: 0.9289
  Novo melhor modelo encontrado! Val Loss: 0.3977. Salvando estado...

Tempo total de treinamento: 9379.961 segundos
✅ Treinamento concluído. O melhor estado do modelo foi salvo na memória.
Melhor modelo salvo em 'scface_best_model.pth'


# 5. Avaliação

## 5.1 Funções auxiliares

In [ ]:
def extract_embeddings_fn(dataloader, model, device):
  """Função responsável por extrair os embeddings de imagens

  Args:
    dataloader (torch.utils.data.Dataloader): Dataloader que contém os dados
    sobre as imagens.
    model (torch.nn.Module): Modelo que gera os embeddings a partir das imagens.
    device (torch.device): Dispositivo no qual as inferências serão executadas.
    Sendo 'cpu' ou 'cuda'

  Returns:
    tuple (np.ndarray, np.darray, list[str]):
      - embeddings (np.ndarray): Lista de embeddings normalizados.
      - labels (np.ndarray): Lista com os rótulos das amostras.
      - paths (list[str]]): Lista com os caminhos originais das imagens.

  """

  model.eval()

  embeddings_list, labels_list, paths_list = [], [], []

  with torch.inference_mode():
    for imgs, labels, paths in tqdm(dataloader, desc="Extraindo embeddings"):
      # Pula batches vazios se o collate_fn filtrou tudo
      if imgs is None:
        continue

      imgs = imgs.to(device)

      embeddings_raw = model(imgs)

      # Normalização L2 explícita dos embeddings
      embeddings = F.normalize(embeddings_raw, p=2, dim=1)

      embeddings_list.append(embeddings.cpu().numpy())

      labels_list.extend(labels.cpu().numpy())

      paths_list.extend(list(paths))

  return np.vstack(embeddings_list), np.array(labels_list), list(paths_list)

In [ ]:
def evaluate_full_metrics(gallery_embeds,
                          gallery_lbls,
                          probe_embeds_full,
                          probe_lbls_full,
                          k_ranks=[1, 5, 10, 20]):
  """
  Calcula as métricas completas relacionadas ao reconhecimento facial.
  Obtém: Rank-k, Acurácia, Precisão, Recall e F1-Score.

  Args:
    gallery_embeds (np.ndarray): Embeddings da galeria.
    gallery_lbls (np.ndarray): Rótulos da galeria.
    probe_embeds_full (np.ndarray): Embeddings das imagens de consulta.
    probe_lbls_full (np.ndarray): Rótulos das imagens de consulta.
    k_ranks (list[int], opcional): Lista de valores k para cálculo do
    Rank-k (padrão: [1, 5, 10, 20]).

  Returns:
    dict: Dicionário com as chaves:
      - `'rank_accuracies'` (dict): Acurácia para cada k em `k_ranks`.
      - `'accuracy'` (float): Acurácia global para Rank-1.
      - `'precision'` (float): Precisão ponderada para Rank-1.
      - `'recall'` (float): Recall ponderado para Rank-1.
      - `'f1_score'` (float): F1-score ponderado para Rank-1.

  """

  num_probes_total = len(probe_lbls_full)

  correct_at_k = {k: 0 for k in k_ranks}

  # Lista para guardar todas as predições de Rank-1
  all_rank1_predictions = []

  print("Iniciando avaliação final...")

  distances = cosine_similarity(probe_embeds_full, gallery_embeds)

  sorted_pred_indices = np.argsort(-distances, axis=1)

  # Itera sobre cada sonda para calcular o Rank-k e coletar a predição Rank-1
  for i in range(num_probes_total):
    true_probe_label = probe_lbls_full[i]

    # Coleta a predição de Rank-1 para as métricas de classificação
    rank1_pred_label = gallery_lbls[sorted_pred_indices[i, 0]]

    all_rank1_predictions.append(rank1_pred_label)

    # Lógica para o Rank-k
    sorted_predicted_labels = gallery_lbls[sorted_pred_indices[i]]

    matches = np.where(sorted_predicted_labels == true_probe_label)[0]

    if len(matches) > 0:
      first_match_rank = matches[0] + 1

      for k in k_ranks:
        if first_match_rank <= k:
          correct_at_k[k] += 1

  print("Processamento concluído.")

  # Cálculo das Métricas

  # Rank-k
  rank_accuracies = {k: count / num_probes_total for k, count in correct_at_k.items()}

  # Métricas de Classificação (baseadas nas predições de Rank-1)
  y_true = probe_lbls_full
  y_pred = np.array(all_rank1_predictions)

  accuracy = accuracy_score(y_true, y_pred)

  precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)

  recall = recall_score(y_true, y_pred, average='weighted')

  f1 = f1_score(y_true, y_pred, average='weighted')

  # Organiza todos os resultados em um único dicionário
  final_results = {
      'rank_accuracies': rank_accuracies,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1_score': f1
  }

  return final_results

In [ ]:
# Preparar o modelo e os dataloaders para avaliação
print("Preparando para a avaliação final...")

final_model = FaceRecognitionModelWithArcFace(num_classes=NUM_SCFACE_CLASSES).to(device)

final_model.load_state_dict(torch.load('scface_best_model.pth',
                                       map_location=device,
                                       weights_only=True))

model_for_embeddings = final_model.backbone

model_for_embeddings.eval()

Preparando para a avaliação final...


InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

## 5.2 Execução da avaliação

In [ ]:
# 1. Extraindo embeddings
print("\nExtraindo embeddings da galeria e sondas do Scface...")

gallery_embeddings, gallery_labels, gallery_paths = extract_embeddings_fn(gallery_loader_for_eval, model_for_embeddings, device)

probe_embeddings, probe_labels, probe_paths = extract_embeddings_fn(probe_loader, model_for_embeddings, device)


Extraindo embeddings da galeria e sondas do Scface...


Extraindo embeddings:   0%|          | 0/37 [00:00<?, ?it/s]

Extraindo embeddings:   0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
# 2. Avaliando performance
print("\nCalculando métricas de performance...")

final_metrics = evaluate_full_metrics(
    gallery_embeds=gallery_embeddings,
    gallery_lbls=gallery_labels,
    probe_embeds_full=probe_embeddings,
    probe_lbls_full=probe_labels
)


Calculando métricas de performance...
Iniciando avaliação final...
Processamento concluído.


In [ ]:
# Imprmindo Rank-k
print("\nMétricas de Identificação (Rank-k):")

for k, acc in final_metrics['rank_accuracies'].items():
  print(f"  - Rank-{k} Accuracy: {acc * 100:.2f}%")

# Imprimindo Métricas de Classificação
print("\nMétricas de Classificação (baseadas no Rank-1):")
print(f"  - Acurácia: {final_metrics['accuracy'] * 100:.2f}%")
print(f"  - Precisão (Ponderada): {final_metrics['precision']:.4f}")
print(f"  - Recall (Ponderado):   {final_metrics['recall']:.4f}")
print(f"  - F1-Score (Ponderado): {final_metrics['f1_score']:.4f}")


Métricas de Identificação (Rank-k):
  - Rank-1 Accuracy: 52.16%
  - Rank-5 Accuracy: 55.85%
  - Rank-10 Accuracy: 62.24%
  - Rank-20 Accuracy: 69.62%

Métricas de Classificação (baseadas no Rank-1):
  - Acurácia: 52.16%
  - Precisão (Ponderada): 0.6092
  - Recall (Ponderado):   0.5216
  - F1-Score (Ponderado): 0.5197


## 5.3 Verificação do tempo de inferência

### 5.3.1 Imagem Única

In [ ]:
def measure_single_image_inference_time(model,
                                        device,
                                        num_runs=100,
                                        num_warmup=10):
  """
  Mede o tempo de inferência para uma única imagem usando um modelo,
  incluindo o cálculo do intervalo de confiança de 95%.

  Args:
    model (torch.nn.Module): Modelo Pytorch a ser avaliado.
    device (torch.device): Dispositivo no qual o modelo será executado.
    num_runs (int): Número de inferências (padrão:100)
    num_warmpu (int): Número de inferências de aquecimento (padrão:10)

  Returns:
    mean_time (float): Tempo médio de inferência para uma imagem.
    std_dev (float): Desvio padrão do mean_time.
  """
  model.eval()

  # Cria uma imagem de entrada de exemplo com o tamanho correto
  dummy_image = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)

  timings = []

  print(f"--- Iniciando medição de inferência para imagem única ---")
  print(f"Executando {num_warmup} passadas de aquecimento (warm-up)...")

  # 1. Aquecimento (Warm-up)
  with torch.inference_mode():
    for warm_up in range(num_warmup):
      warm_up = model(dummy_image)

  # Garante que todas as operações de aquecimento terminaram
  # antes de começar a medir
  if device == 'cuda':
    torch.cuda.synchronize()

  print(f"Executando {num_runs} passadas cronometradas...")

  # 2. Medição
  with torch.inference_mode():
    for i in range(num_runs):
        if device == 'cuda':
          torch.cuda.synchronize()

        start_time = time.perf_counter()

        runs = model(dummy_image)

        if device == 'cuda':
          torch.cuda.synchronize()

        end_time = time.perf_counter()

        # Convertendo para milissegundos (ms)
        timings.append((end_time - start_time) * 1000)

  # 3. Cálculo das estatísticas
  mean_time = np.mean(timings)
  std_dev = np.std(timings)

  # Cálculo do intervalo de confiança de 95%
  confidence = 0.95
  n = len(timings)
  se = std_dev / np.sqrt(n) # Erro padrão da média
  # Usamos a distribuição-t pois temos uma amostra finita
  h = se * stats.t.ppf((1 + confidence) / 2., n - 1)

  confidence_interval_lower = mean_time - h
  confidence_interval_upper = mean_time + h

  print("\n--- Resultados (Imagem Única) ---")
  print(f"Tempo médio de inferência: {mean_time:.2f} ms")
  print(f"Desvio padrão: {std_dev:.2f} ms")
  print(f"Intervalo de Confiança (95%): [{confidence_interval_lower:.2f} ms, {confidence_interval_upper:.2f} ms]")

  return mean_time, std_dev

### 5.3.2 Várias imagens (Throughput)

In [ ]:
def measure_batch_inference_throughput(model, dataloader, device):
  """
  Mede o tempo total para processar um dataloader e calcula
  a vazão (imagens/segundo) de um modelo.

  Args:
    model (torch.nn.Module): Modelo Pytorch a ser avaliado.
    dataloader (torch.utils.data.Dataloader): Dataloader que contém os dados
    sobre as imagens.
    device (torch.device): Dispositivo no qual o modelo será executado.

  Returns:
    images_per_second (images_per_second): Quantidade média de imagens processa
    das por segundo.
  """

  model.eval()

  total_images = len(dataloader.dataset)

  print(f"\n--- Iniciando medição de throughput em {total_images} imagens ---")

  # Aquecimento com o primeiro batch
  try:
    first_batch, x, y = next(iter(dataloader))

    with torch.inference_mode():
      warm_up = model(first_batch.to(device))

    if device == 'cuda':
      torch.cuda.synchronize()

    print("Aquecimento concluído.")
  except StopIteration:
    print("DataLoader está vazio. Não é possível medir.")

    return

  # Medição do tempo total
  start_time = time.perf_counter()

  with torch.inference_mode():
    for images, x, y in tqdm(dataloader, desc="Processando batches"):
      if images is not None:
        runs = model(images.to(device))

  if device == 'cuda':
    torch.cuda.synchronize()

  end_time = time.perf_counter()

  total_time = end_time - start_time
  images_per_second = total_images / total_time
  time_per_image = (total_time / total_images) * 1000

  print("\n--- Resultados (Várias Imagens) ---")
  print(f"Tempo total para processar {total_images} imagens: {total_time:.2f} segundos")
  print(f"Throughput (Vazão): {images_per_second:.2f} imagens por segundo")
  print(f"Tempo médio por imagem em batch: {time_per_image:.2f} ms")

  return images_per_second

In [ ]:
if 'model_for_embeddings' in locals():
  measure_single_image_inference_time(model_for_embeddings, device)
else:
  print("O seu modelo não foi declarado, tente novamente!")

--- Iniciando medição de inferência para imagem única ---
Executando 10 passadas de aquecimento (warm-up)...
Executando 100 passadas cronometradas...

--- Resultados (Imagem Única) ---
Tempo médio de inferência: 18.58 ms
Desvio padrão: 0.19 ms
Intervalo de Confiança (95%): [18.54 ms, 18.62 ms]


In [ ]:
if 'probe_loader' in locals():
  measure_batch_inference_throughput(model_for_embeddings, probe_loader, device)
else:
  print("O seu Dataloader de probes não foi inicializado, tente novamente!")


--- Iniciando medição de throughput em 2800 imagens ---
Aquecimento concluído.


Processando batches:   0%|          | 0/88 [00:00<?, ?it/s]


--- Resultados (Várias Imagens) ---
Tempo total para processar 2800 imagens: 100.73 segundos
Throughput (Vazão): 27.80 imagens por segundo
Tempo médio por imagem em batch: 35.97 ms
